# NEATmap Tutorial

This [notebook](NEATmap.ipynb) is a tutorial to run the main processing script that qunatfies immediate early gene expression from lightsheet data of VISoR ceared brains

In [ ]:
__author__    = 'Weijie Zheng <wjzheng.ahu@gmail.com>'
__license__   = 'GPLv3 - GNU General Pulic License v3'
__copyright__ = 'Copyright © 2023 by Weijie Zheng'

## Initialization

### Initialize NEATmap

In [1]:
# VISoRMap path
import sys
sys.path.append(r'R:\WeijieZheng\VISoRMap_Code\VISoRMap')
#load VISoRMap modules and parameters
from Environment import *
from Parameter import *

## Data preprocessing
### Volume images
VISoR reconstruction of the generated BrainImage path with 561nm channels and 488nm channels to synthesize 3D images with (z, x, y) size (64,3500,2500).

In [ ]:
## 561 channel
root = Data_root
save_brain3d_path = os.path.join(Data_root, 'brain_image_64_' + Channels['staining'])
json_path = os.path.join(Raw_data_root, '..', 'freesia_4.0_C2_488nm_10X.json')
with open(json_path) as f:
    brain = json.load(f)
    images = brain['images']
    total_num = len(images)
b2t3.brain2dto3d(Raw_data_root, save_brain3d_path, total_num, z_num=Preprocessing['z_num'], channel_index=Channels['561nm_index'])

In [ ]:
## 488 channel
save_brain3d_path = os.path.join(Data_root, 'brain_image_64_' + Channels['autofluo'])
b2t3.brain2dto3d(Raw_data_root, save_brain3d_path, total_num, z_num=Preprocessing['z_num'], channel_index=Channels['488nm_index'])

The output is a volumetric image of the labeled signal channel and the autofluorescence channel. For input into the neural network, we set the thickness to 256 µm.
If the thickness value is changed, modify the parameter `Preprocessing['z_num']`.
* `Raw_data_root` is the path to get the data after reconstruction.
* `Data_root` is the path to the workspace.
* `channel_index` is the channel index corresponding to the labeled signal channel and the autofluorescence channel.

### Image clipping
Synthetic 3d brain slices were clip into 70 patches. If you get the training data, you need to input the parameters train_path and label_path.

In [ ]:
## Cut 561nm channel
cut.cut(root=Data_root, cut_size=Preprocessing['cut_size'], channel=Channels['staining'],
cut_index_x=Preprocessing['cut_index_x'], cut_index_y=Preprocessing['cut_index_y'], 
patch_weight_num=Preprocessing['patch_weight_num'], patch_hegiht_num=Preprocessing['patch_height_num'])

In [ ]:
## Cut 488nm channel
cut.cut(root=Data_root, cut_size=Preprocessing['cut_size'], channel=Channels['autofluo'],
cut_index_x=Preprocessing['cut_index_x'], cut_index_y=Preprocessing['cut_index_y'], 
patch_weight_num=Preprocessing['patch_weight_num'], patch_hegiht_num=Preprocessing['patch_height_num'])

Since the size of the volume image is too large to be fed into the neural network, it is clipped into multiple subvolume images of size (64, 256, 256).
* `cut_size` indicates the width and height of the clipped image.
* `cut_index_x` denotes the x coordinate of the initial position of the clip.
* `cut_index_y` denotes the y coordinate of the initial position of the clip.
* `patch_weight_num` indicates that there are multiple patches in the width direction.
* `patch_height_num` indicates that there are multiple patches in the height direction.

[ImageJ](https://imagej.net/Fiji) shows clipped subvolume images of the 561 nm channel.

![Clipping.png](..\Docs\Figure\clipping.PNG)

### Spot segmentation
Get the pixel-level labels of the training data.

In [ ]:
image_path = Train_config['train_image_path']
label_path = Train_config['train_label_path']
sps.spot_seg(train_image_path=image_path, save_seg_path=label_path, channel=Channels['staining'], 
        scaling_param=Spot_seg_config['scaling_param'], wrapper_param=Spot_seg_config['wrapper_param'])

Rerference. Chen, J. et al. The Allen Cell and Structure Segmenter: a new open source toolkit for segmenting 3D intracellular structures in fluorescence microscopy images. 

[Article](http://biorxiv.org/lookup/doi/10.1101/491035).
* `scaling_param` has two options: two values, say `[A, B]`, or single value, say `[K]`. For the first case, `A` and `B` are non-negative values indicating that the full intensity range of the stack will first be cut-off into **[mean - A * std, mean + B * std]** and then rescaled to **[0, 1]**. The smaller the values of `A` and `B` are, the higher the contrast will be. For the second case, `K`>0 indicates min-max Normalization with an absolute intensity upper bound `K` (i.e., anything above `K` will be chopped off and reset as the minimum intensity of the stack) and `K`=0 means min-max Normalization without any intensity bound.
* `wrapper_param`: `[[scale_1, cutoff_1], [scale_2, cutoff_2], ....]` 
    * `scale_x` is set based on the estimated radius of your target dots. For example, if visually the diameter of the dots is usually 3~4 pixels, then you may want to set `scale_x` as `1` or something near `1` (like `1.25`). Multiple scales can be used, if you have dots of very different sizes.  
    * `cutoff_x` is a threshold applied on the actual filter reponse to get the binary result. Smaller `cutoff_x` may yielf more dots and fatter segmentation, while larger `cutoff_x` could be less permisive and yield less dots and slimmer segmentation. 

The [ImageJ](https://imagej.net/Fiji) display of spot segmentation result clipping.

![spot_segmentation_clipped.png](..\Docs\Figure\spot_seg_clip.png)

### Prepare the training data.
Images and labels make up the training data.

In [ ]:
save_path = os.path.join(Train_config['train_data_path'], 'train_data')
tn.get_train_data(image_path, label_path, save_path)

Write the training data name.

In [ ]:
wt.data_list(train_data=Network['train_data'], ratio=Network['train_valid_ratio'], test_id=Test_config['test_id'],
        save_list=Network['save_data_list'], test_singel=Network['test_slice'], test_whole_brain=Network['test_whole_brain'])

Note: Parameters `train_data`, `test_single` and `test_whole_brain` correspond to three cases: training a neural network, testing a single brain slice and automatic whole-brain segmentation, respectively.

## Training neural networks

![3D-HRFormer.png](..\Docs\Figure\model.PNG)

Importing Neural Networks

In [2]:
from Network.model.swin_transform import swin_tiny_patch4_window8_256
# from Network.model.FCN_resnet3d import fcn_resnet34
# from Network.model.unet3d import UNet3D
# from Network.model.ASPP_unet3d import UNet3D
# from Network.model.FCN import FCN
from Network.utils import weights_init

Whether to perform transfer learning or not, the default parameter is False.

In [ ]:
pre_train = Network['pre_train']
checkpoint_path = os.path.join(pre_train, 'epoch_9.pth')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
net = swin_tiny_patch4_window8_256(in_channels=Network['swin_in_channels'], num_classes=Network['num_classes']).to(device)
# net = FCN(in_channels=1, num_classes=Network['num_classes']).to(device)
# net = fcn_resnet34(num_classes=Network['num_classes']).to(device)
# net = UNet3D(in_channel=1, n_classes=Network['num_classes']).to(device)
# net = HRNet(num_classes=Network['num_classes']).to(device)
net = net.apply(weights_init)
print('net initialization succeeds !')
train.train(net, checkpoint_path, pre_train, device)

Neural Networks: **3D-HSFormer**, **FCN**, **ResNet34**, **UNet3D**, **HRNet**
* `in_channels`: The input is a grayscale image and the default number of channels is 1.
* `num_classes`: The input training labels are binarized images with a number of labels of 2.

### Preparing test data

In [ ]:
patch_num = Preprocessing['patch_weight_num'] * Preprocessing['patch_height_num']
ths.slice_test_data(image_patch_path=Test_config['test_image_path'], label_patch_path=Test_config['test_label_path'], 
                channel=Channels['staining'], save_test_path=Test_config['test_save_path'], patch_num=patch_num, 
                test_id=Test_config['test_id'])

### Testing the performance of the network.
Initialization parameters

In [ ]:
from Network.dataset_VISoR import VISoR_dataset
parser = argparse.ArgumentParser()
parser.add_argument('--volume_path', type=str,
                    default='', help='root dir for validation volume data') 
parser.add_argument('--dataset', type=str,
                    default='VISoR', help='experiment_name')
parser.add_argument('--max_epochs', type=int,
                    default=Network['max_epochs'], help='maximum epoch number to train')
parser.add_argument('--num_classes', type=int,
                    default=Network['num_classes'], help='output channel of network')
parser.add_argument('--list_dir', type=str,
                    default='./lists', help='list dir')
parser.add_argument('--model_name', type=str,
                    default='swin', help='model_name')

parser.add_argument('--batch_size', type=int, default=Network['test_batch_size'], help='batch_size per gpu')
parser.add_argument('--img_size', type=int, default=Preprocessing['cut_size'], help='input patch size of network input')
# parser.add_argument('--z_spacing', type=int, default=Network['z_spacing'], help='z spacing')
parser.add_argument('--is_savetif', action="store_true", default=True, help='whether to save results during inference')
parser.add_argument('--test_id', type=int, default=30, help='test the 30th brain slice')
parser.add_argument('--test_save_dir', type=str, default='../predictions', help='saving prediction as tif!')
parser.add_argument('--base_lr', type=float,  default=Network['lr'], help='segmentation network learning rate')
parser.add_argument('--patch_csv_root', type=str,  default=Network['save_patch_metrics'], help='The path to save the patch evaluation result csv')
parser.add_argument('--slice_csv_root', type=str,  default=Network['save_metrics'], help='The path to save the slice evaluation result csv')
args = parser.parse_args(args=[])

dataset_config = {
    'VISoR': {
        'Dataset': VISoR_dataset,
        'volume_path': Test_config['test_save_path'],
        'list_dir': '/home/weijie/3dHRNet/lists',
        'num_classes': Network['num_classes'],
        'z_spacing': Network['z_spacing'],
    },
}
dataset_name = args.dataset
args.exp = dataset_name + str(args.img_size)
args.num_classes = dataset_config[dataset_name]['num_classes']
args.volume_path = dataset_config[dataset_name]['volume_path']
args.Dataset = dataset_config[dataset_name]['Dataset']
args.list_dir = dataset_config[dataset_name]['list_dir']
args.z_spacing = dataset_config[dataset_name]['z_spacing']
args.test_id = Test_config['test_id']
args.is_savetif = True

# name the same snapshot defined in train script!
snapshot_path = Network['snapshot_path']

In [ ]:
# net = HRNet(num_classes=args.num_classes, width=Model['hrnet_width']).cuda()
# net = UNet3D(in_channel=Network['in_channels'], n_classes=args.num_classes).cuda()
# net = FCN(in_channels=Network['in_channels'], num_classes=args.num_classes).cuda()
# net = fcn_resnet34(num_classes=args.num_classes).cuda()
net = swin_tiny_patch4_window8_256(in_channels=Network['swin_in_channels'], num_classes=args.num_classes).cuda()

snapshot = os.path.join(snapshot_path, 'best_model.pth')
if not os.path.exists(snapshot): snapshot = snapshot.replace('best_model', 'epoch_'+str(args.max_epochs-1))
net.load_state_dict(torch.load(snapshot))
snapshot_name = snapshot_path.split('/')[-1]

log_folder = 'test_log/test_log_' + args.exp
os.makedirs(log_folder, exist_ok=True)
logging.basicConfig(filename=log_folder + '/'+snapshot_name+".txt", level=logging.INFO, format='[%(asctime)s.%(msecs)03d] %(message)s', datefmt='%H:%M:%S')
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))
logging.info(str(args))
logging.info(snapshot_name)

if args.is_savenii:
    args.test_save_dir = os.path.join(Network['save_prediction_path'], 'brain_predications_' + args.test_id + '_' + args.model_name + '_epoch' + str(args.max_epochs))
    test_save_path = os.path.join(args.test_save_dir, args.exp, snapshot_name)
    os.makedirs(test_save_path, exist_ok=True)
else:
    test_save_path = None
test.test(args, net, test_save_path)

### Segmentation evaluation
|Evaluation metrics      |Description
|:-----------------------|:--------------------------------------------------------------------------------------------------------------------------------
|Dice coefficient        |The dice coefficient measures the similarity between the predicted segmentation map and the ground-true set of segmentation maps.
|Jaccard coefficient     |The Jaccard coefficient is used to compare the similarities and differences between the predicted set and the ground-truth set.
|Sensitivity             |The sensitivity describes the ratio of all true positives identified to all true positives.

### Automated whole brain c-Fos expression for cellular segmentation.

In [ ]:
from Network.dataset_whole_brain import Whole_brain_dataset
parser = argparse.ArgumentParser()
parser.add_argument('--volume_path', type=str,
                    default='', help='root dir for validation volume data') 
parser.add_argument('--dataset', type=str,
                    default='VISoR', help='experiment_name')
parser.add_argument('--max_epochs', type=int,
                    default=Network['max_epochs'], help='maximum epoch number to train')
parser.add_argument('--num_classes', type=int,
                    default=Network['num_classes'], help='output channel of network')
parser.add_argument('--list_dir', type=str,
                    default='./lists', help='list dir')
parser.add_argument('--model_name', type=str,
                    default='swin', help='model_name')

parser.add_argument('--batch_size', type=int, default=Network['test_batch_size'], help='batch_size per gpu')
parser.add_argument('--img_size', type=int, default=Preprocessing['cut_size'], help='input patch size of network input')
parser.add_argument('--channel', type=str, default=Channels['staining'], help='inference channel')
parser.add_argument('--is_savetif', action="store_true", default=True, help='whether to save results during inference')
parser.add_argument('--test_id', type=int, default=30, help='test the 30th brain slice')
parser.add_argument('--test_save_dir', type=str, default='../predictions', help='saving prediction as tif!')
parser.add_argument('--base_lr', type=float,  default=Network['lr'], help='segmentation network learning rate')
parser.add_argument('--patch_csv_root', type=str,  default=Network['save_patch_metrics'], help='The path to save the patch evaluation result csv')
parser.add_argument('--slice_csv_root', type=str,  default=Network['save_metrics'], help='The path to save the slice evaluation result csv')
args = parser.parse_args(args=[])
data_path = Infer['whole_brain_path']
test_path = os.path.join(data_path, 'PatchImage_' + args.channel)
for ind in range(1, len(os.listdir(test_path)) + 1):
    dataset_config = {
        'VISoR': {
            'Dataset': Whole_brain_dataset,
            'volume_path': test_path + '/patchimage{}'.format(ind),
            'list_dir': Infer['whole_brain_list'],
            'name_list': 'Z{:05d}_test'.format(ind),
            'z_spacing': Network['z_spacing'],
        },
    }
    dataset_name = args.dataset
    args.exp = dataset_name + str(args.img_size)
    args.volume_path = dataset_config[dataset_name]['volume_path']
    args.Dataset = dataset_config[dataset_name]['Dataset']
    args.list_dir = dataset_config[dataset_name]['list_dir']
    args.name_list = dataset_config[dataset_name]['name_list']
    args.z_spacing = dataset_config[dataset_name]['z_spacing']
    
    # name the same snapshot defined in train script!
    snapshot_path = Infer['snapshot_path']
    net = swin_tiny_patch4_window8_256(in_channels=Infer['swin_in_channels'], num_classes=args.num_classes).cuda()

    snapshot = os.path.join(snapshot_path, 'best_model.pth')
    if not os.path.exists(snapshot): snapshot = snapshot.replace('best_model', 'epoch_'+str(args.max_epochs-1))
    net.load_state_dict(torch.load(snapshot))
    snapshot_name = snapshot_path.split('/')[-1]

    log_folder = 'test_log/test_log_' + args.exp
    os.makedirs(log_folder, exist_ok=True)
    logging.basicConfig(filename=log_folder + '/'+snapshot_name+".txt", level=logging.INFO, format='[%(asctime)s.%(msecs)03d] %(message)s', datefmt='%H:%M:%S')
    logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))
    logging.info(str(args))
    logging.info(snapshot_name)

    if args.is_savetif:
        args.test_save_dir = data_path + '/whole_predications_' + args.channel + '/brain_predications_{}_swin_epoch10'.format(ind)
        test_save_path = os.path.join(args.test_save_dir, args.exp, snapshot_name)
        os.makedirs(test_save_path, exist_ok=True)
    else:
        test_save_path = None
    infer_whole.infer_whole_brain(args, net, ind, test_save_path)

## Splice
Automatic segmentation of volume image decomposition into 2-dimensional images.

The segmentation result (z, x, y) is stitched into a volume image of size (64, 3500, 2500).

In [ ]:
channel = Channels['staining']
pred_root = os.path.join(Data_root, 'whole_predications_' + channel)
path = os.path.join(Data_root, 'brain_image_64_' + channel)
save_path_3d = os.path.join(Splicing['save_splicing_path'], 'whole_brain_pred_' + channel)
os.makedirs(save_path_3d, exist_ok=True)
infer_total_num = len(os.listdir(path))
resuidual_z = total_num - (Preprocessing['z_num'] * infer_total_num)
patch_total_num = Preprocessing['patch_weight_num'] * Preprocessing['patch_height_num']
for num in range(1, len(os.listdir(path)) + 1):
    list = rt.load(path, num, total_patch_num=patch_total_num)
    rt.concat(list, save_path_3d, num)
    print('finished {} image '.format(num))
rt.create_residual_image(save_path_3d, infer_total_num + 1, resuidual_z)

## Post-processing
Post-processing of the predicted segmentation results.

In [ ]:
image_path = os.path.join(Data_root, 'brain_image_64_' + Channels['staining'])
path = Splicing['save_splicing_path']
pred_path = os.path.join(path, 'whole_brain_pred_' + Channels['staining'])
path_488nm = os.path.join(path, 'whole_brain_pred_' + Channels['autofluo'])
save_path = os.path.join(path, 'whole_brain_pred_post_filter')
post.spot_filter(image_path, pred_path, path_488nm, save_path, lower_limit=Post['intensity_lower_differ'], 
                    upper_limit=Post['intensity_upper_differ'], min_size=Post['point_min_size'], max_intensity=Post['big_object_size'])

The post-processing process is divided into three main steps:

1. **Autofluorescence filter** is utilized for its predictive segmentation to clear the segmentation of the network predicted autofluorescence signal in the c-Fos channel.
2. **No-soma filter** can remove unreasonably small or large objects from the network prediction segmentation results.
3. **Intensity based prediction result filter** is to remove predicted segmentations with weak c-Fos+ signal intensity (intensity values close to the brain parenchyma signal intensity).
* `lower_limit`  is the lower bound of the difference between the weak signal and the segmentation mask (Mask value = 255).
* `upper_limit`  is the upper bound of the difference between the weak signal and the segmentation mask (Mask value = 255).
* `min_size` is the minimum value of cell size.
* `max_intensity` is the maximum value of the segmented object signal.

Both labeled signal channels and autofluorescent channels underwent the above steps to obtain c-Fos+ segmentation of the whole brain.

Below is an image of [Imaris](http://www.bitplane.com/imaris/imaris) showing the three channels.

![whole_segmentation.png](../Docs/Figure/whole_seg.PNG)

`Red channel`: c-Fos+, `Green channel`: autofluorescent, `Bule channel`: segmentation mask.

## Registration
Reconstructed image voxels (4x4x4 μm) resampled voxels are (25x25x25 μm). 

The reconstructed brain was registered with a reference [autofluorescent brain](http://www.idisco.info) using [Elastix](http://elastix.isi.uu.nl). Transformation parameters of brain alignment were used to obtain the location of cells in the whole brain.

Register to [Allen Brain Atlas CCFv3]((http://atlas.brain-map.org/))

In [ ]:
reconstruction_root = os.path.join(Registration['Raw_brain_path'], 'Reconstruction')
image_list_file = os.path.join(reconstruction_root, 'BrainImage', 'freesia_4.0_C2_488nm_10X.json')
output_path = Registration['output_path'] 
os.makedirs(output_path, exist_ok=True)
template_file = 'Registration/data/ccf_v3_template.json'
output_name = Registration['output_name']
transform_path = os.path.join(reconstruction_root, 'BrainTransform', 'visor_brain.txt')
br.register_brain(image_list_file=image_list_file, output_path=output_path, template_file=template_file, output_name=output_name, 
                brain_transform_path=transform_path)

Use [freesia](https://github.com/BilabOptics/freesia-mapping) to open the **Thumbnail_25.0.json** file generated by the registration program.

![step1.png](../Docs/Figure/freesia_step1.png)

![step2.png](../Docs/Figure/freesia_step2.PNG)

## Sopt mapping

The results of the segmentation were composed into brain slices.

In [ ]:
with tempfile.TemporaryDirectory() as TEMP_PATH:
    splice_path = Splicing['save_splicing_path']
    data_path = os.path.join(splice_path, 'whole_brain_pred_post_filter')
    save_path = os.path.join(TEMP_PATH, 'whole_brain_pred_2d')
    s3t2.seg3d_to_2d(data_path, save_path)
    save_root = os.path.join(splice_path, 'whole_brain_pred_3d')
    st.BrainImage_cat(save_path, save_root, flip=True)

The coordinates of the center of mass of each cell were counted. 

For each c-Fos+ object, the coordinate  values of the three directions of the centroid coordinates 4 × (x, y, z)  were obtained as its physical coordinates

In [ ]:
BrainImage_root = os.path.join(path, 'whole_brain_pred_3d')
csv_root = os.path.join(path, 'whole_brain_cell_counts')
st.BrainImage2Spot(BrainImage_root, csv_root)

Segmentation of cells mapped to the Allen Brain Atlas. (Coronal plane)

In [ ]:
save_csv_root = os.path.join(csv_root, 'Thumbnail_CSV')
total_path = os.path.join(save_csv_root, 'total.txt')
st.Spot_csv(total_path, csv_root, brainimage2d_num=total_num, group_num=Spot_map['group_num'])

`group_num` is the number of `brainimage2d_num` divided by the number of thumbnails (thumbnails are obtained after alignment)

`group_num` = 25 μm / 4 μm = 6.25

Segmentation of cells mapped to the Allen Brain Atlas. (Sagittal plane)

In [ ]:
st.Sagittal_spot_csv(total_path, csv_root, brainimage_x=Brain['weight'], group_num=Spot_map['group_num'])

`brainimage_x` is the width of the reconstructed brain image, the default is 3500.

Segmentation of cells mapped to the Allen Brain Atlas. (Horizontal plane)

In [ ]:
st.Horizontal_spot_csv(total_path, csv_root, brainimage_y=Brain['height'], group_num=Spot_map['group_num'])

`brainimage_y` is the height of the reconstructed brain image, the default is 2500.

The coordinates of the cell mapping positions in the horizontal and sagittal planes at 25 μm can be used to generate horizontal and sagittal cell heat maps for the analysis process.

The location coordinates of the mapped cells were imported using [freesia](https://github.com/BilabOptics/freesia-mapping) to complete cell counts in different brain regions.

1. Import **Spot mapping** to generate **Thumbnail_CSV** folder.

![step3.png](../Docs/Figure/freesia_step3.png)

![out.png](../Docs/Figure/freesia_output1.png)

2. Export cell counting result.

![step4.png](../Docs/Figure/freesia_step4.png)

3. Merger cell counting result.

![step5.png](../Docs/Figure/freesia_step5.png)

Output the **cell-counting.csv** file containing the count results for each brain region.

## Analysis
Remove the cell center-of-mass coordinate points from the freesia-derived image to obtain atlas.

In [ ]:
## freesia export cell counts
image_root = os.path.join(freesia_export_path, 'images')
fb.regenAtlas(image_root)

Obtain the boundaries of the brain atlas.

In [ ]:
atlas_root = os.path.join(image_root, '..', 'atlas')
Edge_root = os.path.join(image_root, '..', 'Edge')
fb.genEdgefromAtlas(atlas_root, Edge_root)

Obtain a brain atlas of the segmented to cells.

In [ ]:
Points_root = os.path.join(image_root, '..', 'Points')
fb.genPoints(image_root,Points_root)  

Obtain segmentation masks and cellular distribution of brain regions of interest.
* `Mask_id` is the pixel value of the brain region annotation.

More pixel values for brain area annotations can be found in the **Parameter.py** file.

In [ ]:
Region_name = 'CBX'
Mask_id = Whole_brain_region_level2[Region_name]#CBX
Region_root = os.path.join(image_root, '..', 'Region')
Points_SingleRegion_root = os.path.join(image_root, '..', 'Points_Single_Region')
fb.genSingleRegion(atlas_root, Region_root, Region_name, Mask_id)
fb.genPointsSingleRegion(image_root, Region_root, Points_SingleRegion_root, Region_name)

Use the surface and spot functions in [Imaris](http://www.bitplane.com/imaris/imaris) to demonstrates the distribution of selected brain regions and c-Fos-expressing cells.

![Region.png](../Docs/Figure/Region_spot.PNG)

The above code inputs the parameter `Whole_brain_reigon_level2` and renders the brain region annotations in combination with the surface function of [Imaris](http://www.bitplane.com/imaris/imaris).

![whole_brain_anno.png](../Docs/Figure/whole_brain_anno.png)

Acquisition of a whole brain cell activity map.

In [ ]:
save_path = os.path.join(Points_root, '..', '3d_result')
save_name = 'whole_brain_point'
altas_num = Preprocessing['brain_total_num'] / Spot_map['group_num']
fb.concat_image(Points_root, save_path, z_num=altas_num, save_name=save_name)

Display the whole brain cell activity map.
* `Atlas_snapshot_id` is the index corresponding to atlas.

In [ ]:
Save_root = Atlas_edge_pionts_save_path
fb.genMergeRaw(Edge_root, Points_root, Save_root, Atlas_snapshot_id)

The number of cells in each layer in different regions of the cerebral cortex was obtained.

In [ ]:
root = Data_root
data_name = Stats['group_data_name']
for k in range(len(data_name)):
    csv_path = os.path.join(root, data_name[k], 'whole_brain_cell_counts')
    ct.cortex_stats(csv_path)
    print('finished {}'.format(data_name[k]))

Access to different areas of the cerebral cortex and c-Fos-expressing cells.
* `get_point`: When False, the generation is a brain area mask. When True, the production is a c-Fos-expressing cell mask.

In [ ]:
root = Data_root
data_name = Stats['Pred_data_list']
for i in range(len(data_name)):
    image_root = os.path.join(root, data_name[i], 'whole_brain_cell_counts/images')
    altas_root =  os.path.join(image_root, '..', 'atlas')
    save_region_root = os.path.join(image_root, '..', 'cortex_region')
    save_points_region_root = os.path.join(image_root, '..', 'points_cortex_region')
    rl.get_multi_region(Isocortex, image_root, altas_root, save_region_root, save_points_region_root, get_point=True)

    save_layer_root = os.path.join(image_root, '..', 'cortex_layer')
    save_points_layer_root = os.path.join(image_root, '..', 'points_cortex_layer')
    rl.get_multi_layer(Isocortex, image_root, altas_root, save_layer_root, save_points_layer_root, get_point=True)
    print('finished {}'.format(data_name[i]))

The spot function of [Imaris](http://www.bitplane.com/imaris/imaris) displays the cellular distribution of c-Fos expression each layer.

![ssp_layer.png](../Docs/Figure/ssp_layer.png)

<font color="#dd0000">**SSp layer 1** </font><br />
<font color="#00dd00">**SSp layer 2/3** </font><br />
<font color="#FB04EA">**SSp layer 4** </font><br />
<font color="#1B8BE4">**SSp layer 5** </font><br />
<font color="#8D03FC">**SSp layer 6** </font><br />

### Volume ratio
Calculated region and cell volume ratio in each layer.

The **volume ratio** computed as (total c-Fos+ neurons volume in the region / region volume) * 100%.

In [ ]:
root = Data_root
data_name = Stats['group_data_name']

for ind in range(len(data_name)):
    region_path = os.path.join(root, data_name[ind], 'whole_brain_cell_counts', 'cortex_region')
    points_label_path = os.path.join(root, data_name[ind], 'whole_brain_cell_counts', 'points_cortex_region')
    save_volume_ratio = os.path.join(region_path, '..')
    vr.get_volume_ratio_csv(Isocortex, region_path, points_label_path, save_volume_ratio)
    print('finished {}'.format(data_name[ind]))

for ind in range(0, len(data_name)):
    path = os.path.join(root, data_name[ind], 'whole_brain_cell_counts')
    layer_path = os.path.join(path, 'cortex_layer')
    layer_points_path = os.path.join(path, 'points_cortex_layer')
    vr.get_whole_layer_volume(Isocortex, layer_path, layer_points_path, savecsv=path)
    print('finished {}'.format(data_name[ind]))

Calculate the mean value of the cell volume ratio in the area and in each layer of the experimental or control group.

In [ ]:
ctrl_name = Stats['Ctrl_data_list']
fs_name = Stats['FST_data_list']
vr.get_whole_layer_mean_volume(root, ctrl_name, Isocortex, save_csv=root, group_name='Ctrl')

### Density
Calculated region and cell density in each layer

The **density** is quantified number of individual c-Fos+ neurou per volume.

In [ ]:
root = Data_root
fs_name = Stats['FST_data_list']
ctrl_name = Stats['Ctrl_data_list']  
data_name = Stats['Pred_data_list']

for ind in range(len(data_name)):
    region_path = os.path.join(root, data_name[ind], 'whole_brain_cell_counts', 'cortex_region')
    csv_path = os.path.join(root, data_name[ind])
    save_density = os.path.join(region_path, '..')
    ds.get_density_csv(region_path, csv_path, Isocortex, save_density)
    print('finished {}'.format(data_name[ind]))

for ind in range(len(data_name)):
    path = os.path.join(root, data_name[ind], 'whole_brain_cell_counts')
    layer_path = os.path.join(path, 'cortex_layer')
    layer_points_path = os.path.join(path, 'points_cortex_layer')
    ds.get_whole_layer_density(Isocortex, root, layer_path, data_name[ind], path)
    print('finished {}'.format(data_name[ind]))

 Calculate the mean value of the cell density in the area and in each layer of the experimental or control group.

In [ ]:
ds.get_whole_layer_mean_density(root, fs_name, Isocortex, save_csv=root, group_name='FS') # group_name = 'FS' or 'Ctrl'

### Intensity

Calculate the intensity values of cell signals in different regions.

* `id` is the serial number of the experimental group or the control group.

In [ ]:
root = Data_root
data_path = os.path.join(root, 'Brainimage_25mic')
label_path = os.path.join(root, '', r'whole_brain_cell_counts\points_cortex_region')
save_csv_root = os.path.join(root, '', r'whole_brain_cell_counts\cortex_intensity')
it.get_intensity_csv(Isocortex, data_path, label_path, save_csv_root, id='Ctrl10')

The percentage of intensity values of cell signals in different regions was calculated.

In [ ]:
intensity_root = os.path.join(Data_root, '', r'whole_brain_cell_counts\cortex_intensity')
savepath = os.path.join(intensity_root, '..', 'intensity_normalized_counts')
it.normalized_counts(Isocortex, intensity_root, savepath)
it.get_whole_norm_counts(path=savepath, region_color=Isocortex, csv_name='Norm_counts.csv')

The intensity values of the cell signals in each subvolume were calculated.
* `min_volume` indicates the signal intensity of the minimum volume of statistical cells.
* `max_volume` indicates the signal intensity of the maximum volume of statistical cells.

In [ ]:
patch_path = ''
it.patch_intensity(data_path=patch_path, label_path=patch_path, save_csv_root=patch_path, min_volume=Stats['min_volume'], max_volume=Stats['max_volume'])

## Statistics
Statistical hypothesis testing of metrics for evaluating DNN segmentation results

In [ ]:
stat.DNN_result_ranksum(data_path=Stats['DNN_results'], model_name=Stats['Model_name'])

The cell count results derived from [freesia](https://github.com/BilabOptics/freesia-mapping) are selected for a particular level of brain area.

In [ ]:
root = Data_root
data_name = Stats['Pred_data_list']
for i in range(0, len(data_name)):
    csv_path = os.path.join(root, data_name[i], 'whole_brain_cell_counts/cell-counting.csv')
    save_path = os.path.join(root, data_name[i], 'whole_brain_cell_counts')
    stat.select_level(csv_path, save_path)
    print('finished {}'.format(data_name[i]))

 The total number of cells in different brain regions or the total number of cells in the left and right parts of different regions were exported.

In [ ]:
for i in range(len(data_name)):
    csv_path = os.path.join(root, data_name[i], 'whole_brain_cell_counts/cell-counts-level6.csv')
    save_path = os.path.join(root, data_name[i], 'whole_brain_cell_counts')
    stat.level6_sum(csv_path, save_path)
    stat.level6_left_right_brain(csv_path, save_path)
    print('finished {}'.format(data_name[i]))

Statistical hypotheses were tested for the number of cells in different brain regions or the number of cells in the left and right parts of different regions.

In [ ]:
fs_name = Stats['FST_data_list']
ctrl_name = Stats['Ctrl_data_list']
stat.ttest_level6(root, fs_name, ctrl_name)
stat.ttest_level6_left_right(root, fs_name, ctrl_name)

Exported statistical hypothesis test signs for the number of cells in the left and right brain halves.
* `select_left`: False -> Statistics right brain
* `select_left`: True -> Statistics left brain

In [ ]:
stat.symbol_left_right_ci(csv_path=root, save_csv=root, select_left=False)

Statistical hypothesis tests were performed on the volume ratio of left and right brain cell halves.

In [ ]:
stat.ttest_multi_region_vol(root, fs_name, ctrl_name)

Results were derived for the number of cells in brain regions at different significant levels.

In [ ]:
symbol = Stats['symbol']
pvals_cutoff = Stats['pvals_cutoff']
for i in range(len(data_name)):
    save_path = os.path.join(root, data_name[i], 'whole_brain_cell_counts/csv')
    os.makedirs(save_path,exist_ok=True) 
    stat.get_multi_cl_counts(root, data_name[i], save_path, symbol, pvals_cutoff)
    print('finished {}'.format(data_name[i]))

Statistical hypothesis testing was performed on the results for different regions of the cerebral cortex and the number of cells each layer.

Results were derived for the number of cells in each layer at different significant levels.

In [ ]:
layer_name = Stats['layer_name']
save_layer_name = Stats['save_layer_name']
stats_dtype = Stats['stats_dtype']
for ln in range(len(layer_name)):
    stat.ttest_multi_layer(root, fs_name, ctrl_name, layer_name[ln], save_layer_name[ln], stats_dtype[2])
    stat.get_layer_ci(root, layer_name[ln], save_layer_name[ln], save_csv=root)
    print(layer_name[ln])

Demonstrate the number of cells in the brain region of interest.

In [ ]:
plot_region_name = Stats['display_region_name']
for i in range(len(data_name)):
    save_path = os.path.join(root, data_name[i], 'whole_brain_cell_counts/csv')
    os.makedirs(save_path,exist_ok=True) 
    stat.get_plot_region_counts(root, data_name[i], plot_region_name, save_path, pvals_cutoff)
    print('finished {}'.format(data_name[i]))

The number of cells in statistical level 2 brain regions.

In [ ]:
stat.level2_count(csv_path=Data_root, save_csv=Data_root)

## Heat map
Heat map of cellular activity in the coronal, horizontal and sagittal planes.
* `fname` is a certain atlas id.csv file in the **Thumbnail_CSV** folder of the experimental and control groups, which records the center-of-mass coordinates of the cells.

In [ ]:
root = Data_root
fname = {'FS8':'0490_25', 'Ctrl8':'0490_25'}
z_length = Preprocessing['brain_total_num'] / Spot_map['group_num']
data_name = [ '', '', ...]
group_name = list(fname.keys())
for i in range(len(data_name)):
    data_path = os.path.join(root, data_name[i], 'coordinate')
    save_path = os.path.join(root, data_name[i], 'cell_density')
    pv.cell_heatmap(data_path, save_path, fname=fname[group_name[i]])
    pv.sagittal_cell_heatmap(data_path, save_path, fname[group_name[i]], z_length)
    pv.horizontal_cell_heatmap(data_path, save_path, fname[group_name[i]], z_length)
    print('finished {}'.format(group_name[i]))

[ImageJ](https://imagej.net/Fiji) displays the cellular thermogram of the coronal plane.

**Left**: forced swimming test group. **Right**: Control group

![heatmap.png](../Docs/Figure/heatmap.PNG)

Calculate the cell density mapped at 25 microns

A total of `35` snapshots were taken, each with an interval atlas id of `16`.

In [ ]:
for i in range(len(data_name)):
    for j in range(35):
        index = j*16
        pv.group_cell_heatmap(data_path=os.path.join(root, data_name[i]), save_path=os.path.join(root, data_name[i], 'group_cell_denisty'), id=index)
        if index >= z_length:
            break
        print('finish {}'.format(index))
        print('finished' + data_name[i])

Whole brain cell heat map

In [ ]:
csv_path = os.path.join(Data_root, '', r'whole_brain_cell_counts\Thumbnail_CSV')
save_path = os.path.join(Data_root, '', 'cell_density')
save_name = ''
pv.whole_brain_cell_heatmap(csv_path, save_path, z_length=len(os.listdir(csv_path)), save_name=save_name)

## P-value map

In [ ]:
group1 = ['', '', ...]
group2 = ['', '', ...]
fs_name = Stats['FST_data_list']
ctrl_name = Stats['Ctrl_data_list']
for i in range(35):
    index = i*16
    group_fs = [os.path.join(root, fs_name[j], 'group_cell_denisty\{}_cell_density.tif'.format(index)) for j in range(len(fs_name))]
    # group1.append(group_fs)
    group_ctrl = [os.path.join(root, ctrl_name[k], 'group_cell_denisty\{}_cell_density.tif'.format(index)) for k in range(len(ctrl_name))]
    # group2.append(group_ctrl)
    save_p_map_path = os.path.join(Data_root, '', 'group_p_map')
    save_name = str(index) + '_p_map' 
    pv.generate_p_mapping(group_fs, group_ctrl, save_p_map_path, save_name)
    print('finished {}'.format(index))

The [ImageJ](https://imagej.net/Fiji) display generates a P-value map.

The **light blue area** indicates a significant difference in the number of regional c-Fos+ cells between the FST group and the control group.

![p_vaule.png](../Docs/Figure/P_value.png)

Calculate the coefficient of variation (CV) and z-score.

In [ ]:
csv_path = Stats['level6_cell_counts_path']
index = Stats['group_index']
cz.coefficient_of_variation(csv_path=csv_path, save_path=csv_path, index=index)
cz.z_score(csv_path=csv_path, save_path=csv_path, index=index)

Calculate the log2(fold change).

In [ ]:
root = Data_root
exp_name = Stats['Exp_data_list'] 
obser_name = Stats['Observe_data_list'] 
con_name = Stats['Ctrl_data_list']
lc.log2Change(root, exp_name, obser_name, con_name) 

Calculate the spearman's correlation.

In [ ]:
data_root = Data_root
save_root = Data_root
sc.spearman_corr(data_root, save_root)